## Techniques for Limiting Messages to Save Tokens

Reducing message history is important for saving tokens and optimizing performance in agentic applications that use LLMs over multi-turn conversations.

In [21]:
from typing import List, Annotated
from langgraph.graph import StateGraph, MessagesState, add_messages, START, END
from langchain_core.messages import (
    HumanMessage, 
    AIMessage, 
    SystemMessage,
    AnyMessage,
    RemoveMessage,
    trim_messages
)
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

True

#### **Initial Setup**

- A single-node workflow is created with:

    - A system message ("You are a FinTech specialist").
    - Few-shot examples of human and AI messages (e.g., "What is Pokémon?" → refusal to answer).


In [22]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [23]:
def llm_node(state: MessagesState):
    return {"messages": llm.invoke(state["messages"])}

In [24]:
workflow = StateGraph(MessagesState)

workflow.add_node("llm_node", llm_node)

workflow.add_edge(START, "llm_node")
workflow.add_edge("llm_node", END)

graph = workflow.compile()



**Useful Messages List**

Each message is assigned a unique **ID** for easier filtering later.

In [25]:
messages = [
    SystemMessage(
        content="You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.",
        name="System",
        id="0",
    ),
    HumanMessage(
        content="What is Pokemon?",
        name="User",
        id="1"
    ),
    AIMessage(
        content="I'm here to provide information specifically about FinTech. If you have " 
                "any questions related to financial technology, such as digital payments, " 
                "blockchain, cryptocurrencies, or financial services innovations, feel free " 
                "to ask!",
        name="FintechAssistant",
        id="2",
    ),
    HumanMessage(
        content="What is BlockChain?",
        name="User",
        id="3"
    ),
    AIMessage(
        content="Blockchain is a decentralized digital ledger technology that records" 
                "transactions across multiple computers in a way that ensures the security, " 
                "transparency, and integrity of the data. Each transaction is grouped into " 
                "a block, and these blocks are linked together in chronological order to form " 
                "a chain, hence the name blockchain.",
        name="FintechAssistant",
        id="4",
    ),
    HumanMessage(
        content="What is a credit card fraud?",
        name="User",
        id="5"
    ),
]

#### **Token Usage Without Trimming**

When invoking with the full message history:
- Prompt tokens and completion tokens are relatively high

In [26]:
complete_output = graph.invoke({"messages": messages})

In [27]:
complete_output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Credit card fraud is a type of financial fraud that involves the unauthorized use of someone else's credit card information to make purchases or access funds. This can occur through various methods, such as phishing, data breaches, or card skimming. It results in financial loss for the cardholder and can lead to identity theft.


In [28]:
complete_output["messages"][-1].response_metadata["token_usage"]

{'completion_tokens': 63,
 'prompt_tokens': 176,
 'total_tokens': 239,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

#### **Simple Manual Filtering**

- Manually invoking the LLM with only a **subset** of the messages (e.g., first and last message).

In [29]:
filtered_output = graph.invoke({"messages": [messages[0], messages[-1]]})

In [30]:
filtered_output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Credit card fraud is the unauthorized use of a credit card or credit card information to make purchases or access funds. This can occur through various methods, such as card theft, phishing, or data breaches, leading to financial loss for the cardholder and the issuing bank.


In [31]:
filtered_output["messages"][-1].response_metadata["token_usage"]

{'completion_tokens': 54,
 'prompt_tokens': 42,
 'total_tokens': 96,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

#### **Filtering Inside the Node**

- A custom state is created by extending `MessageState` , including:

    - `messages`
    - `filtered_messages`

- Inside the node, **only the last three messages** are passed to the LLM.

In [32]:
class State(MessagesState):
    filtered_messages: Annotated[List[AnyMessage], add_messages]

In [33]:
def llm_node(state: State):
    filtered_messages = state["messages"][-3:]
    ai_message = llm.invoke(filtered_messages)
    filtered_messages.append(ai_message)
    return {"messages": ai_message, "filtered_messages": filtered_messages}

In [35]:
workflow = StateGraph(State)
workflow.add_node("llm_node", llm_node)
workflow.add_edge(START, "llm_node")
workflow.add_edge("llm_node", END)
graph = workflow.compile()

In [36]:
for m in messages:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================ Human Message =================================
Name: User

What is Pokemon?
================================== Ai Message ==================================
Name: FintechAssistant

I'm here to provide information specifically about FinTech. If you have any questions related to financial technology, such as digital payments, blockchain, cryptocurrencies, or financial services innovations, feel free to ask!
================================ Human Message =================================
Name: User

What is BlockChain?
================================== Ai Message ==================================
Name: FintechAssistant

Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the securi

In [37]:
output = graph.invoke({'messages': messages})
for m in output['filtered_messages']:
    m.pretty_print()

================================ Human Message =================================
Name: User

What is BlockChain?
================================== Ai Message ==================================
Name: FintechAssistant

Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the security, transparency, and integrity of the data. Each transaction is grouped into a block, and these blocks are linked together in chronological order to form a chain, hence the name blockchain.
================================ Human Message =================================
Name: User

What is a credit card fraud?
================================== Ai Message ==================================

Credit card fraud is a type of financial fraud that involves the unauthorized use of someone else's credit card information to make purchases or access funds. This can occur in various ways, including:

1. **Card Not Present Fraud**: This happens w

#### **Using Remove Messages Strategy**

- The **remove_message** reducer from LangGraph is used to **delete** unwanted messages based on IDs.
- A deletion list is created to filter out irrelevant few-shot examples while preserving essential context.

In [38]:
messages[:-3]

[SystemMessage(content="You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.", name='System', id='0'),
 HumanMessage(content='What is Pokemon?', name='User', id='1'),
 AIMessage(content="I'm here to provide information specifically about FinTech. If you have any questions related to financial technology, such as digital payments, blockchain, cryptocurrencies, or financial services innovations, feel free to ask!", name='FintechAssistant', id='2')]

In [39]:
delete_messages = [RemoveMessage(id=m.id) for m in messages[:-3]]
add_messages(messages, delete_messages)

/var/folders/f3/_6ts5bqd5s30t0mq76kjkdxc0000gn/T/ipykernel_18094/838439161.py:1: LangChainBetaWarning: The class `RemoveMessage` is in beta. It is actively being worked on, so the API may change.
  delete_messages = [RemoveMessage(id=m.id) for m in messages[:-3]]


[HumanMessage(content='What is BlockChain?', name='User', id='3'),
 AIMessage(content='Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the security, transparency, and integrity of the data. Each transaction is grouped into a block, and these blocks are linked together in chronological order to form a chain, hence the name blockchain.', name='FintechAssistant', id='4'),
 HumanMessage(content='What is a credit card fraud?', name='User', id='5')]

In [40]:
class State(MessagesState):
    filtered_messages: Annotated[List[AnyMessage], add_messages]

In [41]:
def removal_filter(state: State):
    filtered_messages = [
        RemoveMessage(id=m.id) 
            for m in state["messages"][:-3] 
            if m.name != "System"
    ]
    return {
        "filtered_messages": add_messages(
            state["messages"], 
            filtered_messages
        )
    }

In [42]:
def llm_node(state: State):
    ai_message = llm.invoke(state["filtered_messages"])
    return {
        "filtered_messages": ai_message,
    }

In [43]:
workflow = StateGraph(State)
workflow.add_node("llm_node", llm_node)
workflow.add_node("removal_filter", removal_filter)
workflow.add_edge(START, "removal_filter")
workflow.add_edge("removal_filter", "llm_node")
workflow.add_edge("llm_node", END)

graph = workflow.compile()


In [44]:
for m in messages:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================ Human Message =================================
Name: User

What is Pokemon?
================================== Ai Message ==================================
Name: FintechAssistant

I'm here to provide information specifically about FinTech. If you have any questions related to financial technology, such as digital payments, blockchain, cryptocurrencies, or financial services innovations, feel free to ask!
================================ Human Message =================================
Name: User

What is BlockChain?
================================== Ai Message ==================================
Name: FintechAssistant

Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the securi

In [45]:
output = graph.invoke({'messages': messages})
for m in output['filtered_messages']:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================ Human Message =================================
Name: User

What is BlockChain?
================================== Ai Message ==================================
Name: FintechAssistant

Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the security, transparency, and integrity of the data. Each transaction is grouped into a block, and these blocks are linked together in chronological order to form a chain, hence the name blockchain.
================================ Human Message =================================
Name: User

What is a credit card fraud?
================================== Ai Message ==================================

Credit card fraud is a type of financial crime t

#### **Using Trim Messages for Token Limits**

- Trim strategy is introduced to limit messages by token budget:

    - Keeps only the latest messages that fit within a specified token limit.

- The `trim_messages()` method is used with different max token thresholds.

In [46]:
trim_messages(
    messages,
    max_tokens=30,
    strategy="last",
    token_counter=llm,
    allow_partial=False,
    include_system=True
)

[SystemMessage(content="You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.", name='System', id='0')]

In [47]:
class State(MessagesState):
    max_tokens: int
    filtered_messages: Annotated[List[AnyMessage], add_messages]

In [50]:
def trim_filter(state: State):
    max_tokens = state["max_tokens"]
    messages = state["messages"]
    filtered_messages = messages
    if max_tokens:
        filtered_messages = trim_messages(
            messages=messages,
            max_tokens=max_tokens,
            strategy="last",
            token_counter=llm,
            include_system=True,
            allow_partial=False
        )
    return {"filtered_messages": filtered_messages}

In [51]:
def llm_node(state: State):
    return {"filtered_messages": llm.invoke(state["filtered_messages"])}

In [52]:
workflow = StateGraph(State)
workflow.add_node("llm_node", llm_node)
workflow.add_node("trim_filter", trim_filter)
workflow.add_edge(START, "trim_filter")
workflow.add_edge("trim_filter", "llm_node")
workflow.add_edge("llm_node", END)

graph = workflow.compile()

In [53]:
for m in messages:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================ Human Message =================================
Name: User

What is Pokemon?
================================== Ai Message ==================================
Name: FintechAssistant

I'm here to provide information specifically about FinTech. If you have any questions related to financial technology, such as digital payments, blockchain, cryptocurrencies, or financial services innovations, feel free to ask!
================================ Human Message =================================
Name: User

What is BlockChain?
================================== Ai Message ==================================
Name: FintechAssistant

Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the securi

In [54]:
output = graph.invoke(
    input={
        "max_tokens": 50,
        "messages": messages
    }
)

In [55]:
for m in output['filtered_messages']:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================ Human Message =================================
Name: User

What is a credit card fraud?
================================== Ai Message ==================================

Credit card fraud is the unauthorized use of a credit card or credit card information to make purchases or access funds. This can occur through various methods, including card theft, phishing, data breaches, or skimming devices. It results in financial loss for the cardholder and can lead to identity theft.


#### **Summarization to Compress Messages**

- Messages between the initial system message and the latest user query are **summarized**.
- Summarization is prompted by inserting a special **HumanMessage**: "Summarize the above conversation."
- This produces a concise summary that replaces multiple older turns.

In [56]:
messages[1:-1]

[HumanMessage(content='What is Pokemon?', name='User', id='1'),
 AIMessage(content="I'm here to provide information specifically about FinTech. If you have any questions related to financial technology, such as digital payments, blockchain, cryptocurrencies, or financial services innovations, feel free to ask!", name='FintechAssistant', id='2'),
 HumanMessage(content='What is BlockChain?', name='User', id='3'),
 AIMessage(content='Blockchain is a decentralized digital ledger technology that recordstransactions across multiple computers in a way that ensures the security, transparency, and integrity of the data. Each transaction is grouped into a block, and these blocks are linked together in chronological order to form a chain, hence the name blockchain.', name='FintechAssistant', id='4')]

In [57]:
messages_to_summarize = messages[1:-1]
summary_message = HumanMessage(
    content="Create a summary of the conversation above:", 
    name="User"
)
ai_message = llm.invoke(
    add_messages(
        messages_to_summarize,
        summary_message
    )
)

In [58]:
ai_message.content

'The user initially asked about Pokémon, but the assistant redirected the conversation to focus on FinTech topics. The user then inquired about blockchain, to which the assistant provided a brief explanation, describing blockchain as a decentralized digital ledger technology that securely records transactions in a chronological chain of blocks.'

In [59]:
ai_message.id = "1"
messages[-1].id = "2"

In [60]:
remaining_messages = [messages[0]] + [ai_message] + [messages[-1]]

In [61]:
remaining_messages

[SystemMessage(content="You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.", name='System', id='0'),
 AIMessage(content='The user initially asked about Pokémon, but the assistant redirected the conversation to focus on FinTech topics. The user then inquired about blockchain, to which the assistant provided a brief explanation, describing blockchain as a decentralized digital ledger technology that securely records transactions in a chronological chain of blocks.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 151, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='1', 

In [62]:
remaining_messages.append(llm.invoke(remaining_messages))

In [63]:
for m in remaining_messages:
    m.pretty_print()

================================ System Message ================================
Name: System

You're a FinTech specialist. You're not allowed to talk about anything else. Be concise in your answers.
================================== Ai Message ==================================

The user initially asked about Pokémon, but the assistant redirected the conversation to focus on FinTech topics. The user then inquired about blockchain, to which the assistant provided a brief explanation, describing blockchain as a decentralized digital ledger technology that securely records transactions in a chronological chain of blocks.
================================ Human Message =================================
Name: User

What is a credit card fraud?
================================== Ai Message ==================================

Credit card fraud is the unauthorized use of a credit card or credit card information to make purchases or access funds. This can occur through various methods, such as

In [64]:
remaining_messages[-1].response_metadata["token_usage"]

{'completion_tokens': 64,
 'prompt_tokens': 102,
 'total_tokens': 166,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}